In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import csv
import io
import itertools
import shutil
import cv2
import pathlib
from zipfile import ZipFile
from PIL import Image
from datetime import datetime

from keras import backend as K
from keras.layers import Layer,InputSpec
import keras.layers as kl

import sklearn.metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from imblearn.over_sampling import RandomOverSampler 

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout, GlobalAveragePooling2D, Activation, MaxPooling2D, concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

In [ ]:
!mkdir /root/.kaggle
!sudo chmod 600 kaggle.json
!mv kaggle.json /root/.kaggle

In [ ]:
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000

100% 5.19G/5.20G [01:15<00:00, 99.9MB/s]
100% 5.20G/5.20G [01:15<00:00, 74.0MB/s]


In [ ]:
with ZipFile('skin-cancer-mnist-ham10000.zip', 'r') as zip_ref:
  zip_ref.extractall()

In [ ]:
ham_part_1 = 'HAM10000_images_part_1'
ham_part_2 = 'HAM10000_images_part_2'

part_1_imgs = os.listdir(ham_part_1)
part_2_imgs = os.listdir(ham_part_2)

In [ ]:
_2020_pd = pd.read_csv('D:/proj_img/jpeg-melanoma-384x384/train.csv')
_2020_pd['diagnosis'].value_counts()

unknown                               27124
nevus                                  5193
melanoma                                584
seborrheic keratosis                    135
lentigo NOS                              44
lichenoid keratosis                      37
solar lentigo                             7
cafe-au-lait macule                       1
atypical melanocytic proliferation        1
Name: diagnosis, dtype: int64

In [ ]:
_2020_pd['diagnosis'] = _2020_pd['diagnosis'].apply(lambda x: x.replace('melanoma', 'mel'))

In [ ]:
_2020_pd['diagnosis'].value_counts()

unknown                               27124
nevus                                  5193
mel                                     584
seborrheic keratosis                    135
lentigo NOS                              44
lichenoid keratosis                      37
solar lentigo                             7
cafe-au-lait macule                       1
atypical melanocytic proliferation        1
Name: diagnosis, dtype: int64

In [ ]:
mel_pd = _2020_pd[_2020_pd['diagnosis'] == 'mel']

In [ ]:
mel_pd.to_csv('D:/proj_img/mel_pd.csv')

In [ ]:
_2018_pd = pd.read_csv('/Users/user/Desktop/b21_proj/Dataset/HAM10000_metadata.csv')
_2018_pd['dx'].value_counts()

nv       6705
mel      1697
bkl      1099
bcc       514
akiec     327
vasc      142
df        115
Name: dx, dtype: int64

In [ ]:
df_count = meta_data.groupby('lesion_id').count()
df_count.head()

,image_id,dx,dx_type,age,sex,localization
lesion_id,,,,,,
HAM_0000000,2,2,2,2,2,2
HAM_0000001,1,1,1,1,1,1
HAM_0000002,3,3,3,3,3,3
HAM_0000003,1,1,1,1,1,1
HAM_0000004,1,1,1,1,1,1


In [ ]:
df_count = df_count[df_count['dx'] == 1]
df_count.reset_index(inplace=True)

In [ ]:
def duplicates(x):
    unique = set(df_count['lesion_id'])
    if x in unique:
        return 'no' 
    else:
        return 'duplicates'

In [ ]:
_2018_pd['is_duplicate'] = _2018_pd['lesion_id'].apply(duplicates)
_2018_pd.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,is_duplicate
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern,duplicates
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,duplicates
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,duplicates
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern,duplicates
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern,duplicates


In [ ]:
df_count = _2018_pd[_2018_pd['is_duplicate'] == 'no']

In [ ]:
df_count['dx'].value_counts()

nv       4415
bkl       440
mel       230
bcc       175
akiec     151
vasc       64
df         39
Name: dx, dtype: int64

In [ ]:
train, test = train_test_split(_2018_pd, train_size=0.85, stratify=_2018_pd['dx'])

In [ ]:
train['dx'].value_counts()
train_proportion = [x for x in train['dx'].value_counts() / train['dx'].count()]

train_proportion

[0.6325896325896326,
 0.16006216006216006,
 0.10367410367410368,
 0.048507048507048504,
 0.03085803085803086,
 0.013431013431013432,
 0.010878010878010878]

In [ ]:
test['dx'].value_counts()
test_proportion = [x for x in test['dx'].value_counts() / test['dx'].count()]

test_proportion

[0.6327044025157232,
 0.16037735849056603,
 0.10377358490566038,
 0.048427672955974846,
 0.030817610062893082,
 0.013207547169811321,
 0.010691823899371069]

In [ ]:
def split_data(x):
  train_data = set(train['image_id'])
  if str(x) in train_data:
    return 'train'
  else:
    return 'test'

_2018_pd['splits'] = _2018_pd['image_id'].apply(split_data)

In [ ]:
train_list = list(train['image_id'])
test_list = list(test['image_id'])

len(test_list)

1590

In [ ]:
len(train_list)

9009

In [ ]:
for row in _2018_pd.itertuples():
  print(row)

In [ ]:
mel_pd

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target,tfrecord,width,height
91,ISIC_0149568,IP_0962375,female,55.0,upper extremity,mel,malignant,1,0,6000,4000
235,ISIC_0188432,IP_0135517,female,50.0,upper extremity,mel,malignant,1,5,3264,2448
314,ISIC_0207268,IP_7735373,male,55.0,torso,mel,malignant,1,3,6000,4000
399,ISIC_0232101,IP_8349964,male,65.0,torso,mel,malignant,1,9,4288,2848
459,ISIC_0247330,IP_3232631,female,65.0,lower extremity,mel,malignant,1,10,6000,4000
...,...,...,...,...,...,...,...,...,...,...,...
32969,ISIC_9955163,IP_7507212,male,55.0,upper extremity,mel,malignant,1,5,640,480
33000,ISIC_9963177,IP_1165806,male,70.0,torso,mel,malignant,1,13,1872,1053
33014,ISIC_9967383,IP_7887363,male,60.0,upper extremity,mel,malignant,1,-1,6000,4000
33050,ISIC_9978107,IP_2860540,male,65.0,lower extremity,mel,malignant,1,6,4288,2848


In [ ]:
all_image_dir = 'C:/Users/user/Desktop/b21_proj/Dataset/all_image/'
mel_2020_dir = 'D:/proj_img/jpeg-melanoma-384x384/train/'

for img in mel_pd.itertuples():
  source = os.path.join(mel_2020_dir, img[1] + '.jpg')
  shutil.copy(source, all_image_dir)

In [ ]:
stratify_split_dir = 'D:/proj_img/2018_image_dir/'
all_image_dir = 'C:/Users/user/Desktop/b21_proj/Dataset/all_image/'

for row in _2018_pd.itertuples():
  image_id = row[2] + '.jpg'
  dx = row[3]
  splits = row[-1]

  imgs_path = os.path.join(all_image_dir, image_id)
  target_path = os.path.join(stratify_split_dir + splits + '/', dx)

  if not os.path.exists(target_path):
    pathlib.Path(target_path).mkdir(parents=True, exist_ok=True)

  shutil.copy(imgs_path, target_path)

In [ ]:
train['dx'].value_counts()

nv       5699
mel      1442
bkl       934
bcc       437
akiec     278
vasc      121
df         98
Name: dx, dtype: int64

In [ ]:
class_name = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

train_dir = 'D:/proj_img/2018_image_dir/train/'
aug_dir_result = 'D:/proj_img/2018_aug_dir/train/'

image_size = 384

for img_class in class_name:
  temp_dir = 'D:/proj_img/temp_img/'
  os.mkdir(temp_dir)

  temp_child_dir = 'D:/proj_img/temp_img/child/'
  os.mkdir(temp_child_dir)

  source_dir = os.path.join(train_dir, img_class)
  target_dir = os.path.join(aug_dir_result, img_class)
  pathlib.Path(target_dir).mkdir(parents=True, exist_ok=True)

  datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=90,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    brightness_range=[0.5, 1.3],
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
  )

  imgs_list = os.listdir(train_dir + img_class)

  for img in imgs_list:
    source = os.path.join(train_dir + img_class + '/', img)
    shutil.copy(source, temp_dir + 'child/')

  batch_size = 50

  aug_generator = datagen.flow_from_directory(
      temp_dir,
      batch_size=batch_size,
      save_to_dir=target_dir,
      save_format='jpg',
      target_size=(image_size, image_size)
  )

  aug_images = 8000

  num_files = len(os.listdir(source_dir))
  num_batches = int(np.ceil((aug_images - num_files) / batch_size))

  for i in range(0, num_batches):
    next(aug_generator)

  shutil.rmtree(temp_dir)

Found 278 images belonging to 1 classes.
Found 437 images belonging to 1 classes.
Found 934 images belonging to 1 classes.
Found 98 images belonging to 1 classes.
Found 1442 images belonging to 1 classes.
Found 5699 images belonging to 1 classes.
Found 121 images belonging to 1 classes.


In [ ]:
#InceptionResNetV2
pre_trained_model = tf.keras.applications.InceptionResNetV2(
    input_shape=(229,229,3),
    include_top=False,
    weights='imagenet',
    classifier_activation='softmax',
    pooling=None
    )

pre_trained_model.summary()

Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 229, 229, 3) 0                                            
__________________________________________________________________________________________________
conv2d_406 (Conv2D)             (None, 114, 114, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_406 (BatchN (None, 114, 114, 32) 96          conv2d_406[0][0]                 
__________________________________________________________________________________________________
activation_406 (Activation)     (None, 114, 114, 32) 0           batch_normalization_406[0][0]    
________________________________________________________________________________

In [ ]:
NUM_EPOCHS = 200
BATCH_SIZE = 32

In [ ]:
version_name = 'InceptionResNetV2_2018_2020_229'

In [ ]:
train_dir = 'D:/proj_img/2018_aug_dir/train/'
test_dir = 'D:/proj_img/2018_aug_dir/test/'
image_size = 229

datagen = ImageDataGenerator()

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(image_size, image_size),
    batch_size=10,
    class_mode='categorical',
    shuffle=True
)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(image_size, image_size),
    batch_size=1,
    class_mode='categorical',
    shuffle=False
)

Found 53691 images belonging to 7 classes.
Found 1590 images belonging to 7 classes.


# Soft Attention

In [ ]:
#Soft Attention

class SoftAttention(Layer):
    def __init__(self,ch,m,concat_with_x=False,aggregate=False,**kwargs):
        self.channels=int(ch)
        self.multiheads = m
        self.aggregate_channels = aggregate
        self.concat_input_with_scaled = concat_with_x

        
        super(SoftAttention,self).__init__(**kwargs)

    def build(self,input_shape):

        self.i_shape = input_shape

        kernel_shape_conv3d = (self.channels, 3, 3) + (1, self.multiheads) # DHWC
    
        self.out_attention_maps_shape = input_shape[0:1]+(self.multiheads,)+input_shape[1:-1]
        
        if self.aggregate_channels==False:

            self.out_features_shape = input_shape[:-1]+(input_shape[-1]+(input_shape[-1]*self.multiheads),)
        else:
            if self.concat_input_with_scaled:
                self.out_features_shape = input_shape[:-1]+(input_shape[-1]*2,)
            else:
                self.out_features_shape = input_shape
        

        self.kernel_conv3d = self.add_weight(shape=kernel_shape_conv3d,
                                        initializer='he_uniform',
                                        name='kernel_conv3d')
        self.bias_conv3d = self.add_weight(shape=(self.multiheads,),
                                      initializer='zeros',
                                      name='bias_conv3d')

        super(SoftAttention, self).build(input_shape)

    def call(self, x):

        exp_x = K.expand_dims(x,axis=-1)

        c3d = K.conv3d(exp_x,
                     kernel=self.kernel_conv3d,
                     strides=(1,1,self.i_shape[-1]), padding='same', data_format='channels_last')
        conv3d = K.bias_add(c3d,
                        self.bias_conv3d)
        conv3d = kl.Activation('relu')(conv3d)

        conv3d = K.permute_dimensions(conv3d,pattern=(0,4,1,2,3))

        
        conv3d = K.squeeze(conv3d, axis=-1)
        conv3d = K.reshape(conv3d,shape=(-1, self.multiheads ,self.i_shape[1]*self.i_shape[2]))

        softmax_alpha = K.softmax(conv3d, axis=-1) 
        softmax_alpha = kl.Reshape(target_shape=(self.multiheads, self.i_shape[1],self.i_shape[2]))(softmax_alpha)

        
        if self.aggregate_channels==False:
            exp_softmax_alpha = K.expand_dims(softmax_alpha, axis=-1)       
            exp_softmax_alpha = K.permute_dimensions(exp_softmax_alpha,pattern=(0,2,3,1,4))
   
            x_exp = K.expand_dims(x,axis=-2)
   
            u = kl.Multiply()([exp_softmax_alpha, x_exp])   
  
            u = kl.Reshape(target_shape=(self.i_shape[1],self.i_shape[2],u.shape[-1]*u.shape[-2]))(u)

        else:
            exp_softmax_alpha = K.permute_dimensions(softmax_alpha,pattern=(0,2,3,1))

            exp_softmax_alpha = K.sum(exp_softmax_alpha,axis=-1)

            exp_softmax_alpha = K.expand_dims(exp_softmax_alpha, axis=-1)

            u = kl.Multiply()([exp_softmax_alpha, x])   

        if self.concat_input_with_scaled:
            o = kl.Concatenate(axis=-1)([u,x])
        else:
            o = u
        
        return [o, softmax_alpha]

    def compute_output_shape(self, input_shape): 
        return [self.out_features_shape, self.out_attention_maps_shape]

    
    def get_config(self):
        return super(SoftAttention,self).get_config()

Without SA

In [ ]:
output_pre_trained = pre_trained_model.layers[377].output
output_pre_trained = Activation('relu')(output_pre_trained)
output_pre_trained = Dropout(0.5)(output_pre_trained)
output_pre_trained = Flatten()(output_pre_trained)
output = Dense(7, activation='softmax')(output_pre_trained)

transfer_model = Model(inputs=pre_trained_model.input, outputs=output)
transfer_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 229, 229, 3) 0                                            
__________________________________________________________________________________________________
conv2d_406 (Conv2D)             (None, 114, 114, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_406 (BatchN (None, 114, 114, 32) 96          conv2d_406[0][0]                 
__________________________________________________________________________________________________
activation_406 (Activation)     (None, 114, 114, 32) 0           batch_normalization_406[0][0]    
____________________________________________________________________________________________

With SA

In [ ]:
output_pre_trained = pre_trained_model.layers[-28].output
attention_layer,map2 = SoftAttention(aggregate=True,m=16,concat_with_x=False,ch=int(output_pre_trained.shape[-1]),name='soft_attention')(output_pre_trained)
attention_layer=(MaxPooling2D(pool_size=(2, 2),padding="same")(attention_layer))
output_pre_trained=(MaxPooling2D(pool_size=(2, 2),padding="same")(output_pre_trained))

output_pre_trained = concatenate([output_pre_trained,attention_layer])
output_pre_trained = Dropout(0.5)(output_pre_trained)
output_pre_trained = Flatten()(output_pre_trained)
output = Dense(7, activation='softmax')(output_pre_trained)

transfer_model = Model(inputs=pre_trained_model.input, outputs=output)
transfer_model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 229, 229, 3) 0                                            
__________________________________________________________________________________________________
conv2d_203 (Conv2D)             (None, 114, 114, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_203 (BatchN (None, 114, 114, 32) 96          conv2d_203[0][0]                 
__________________________________________________________________________________________________
activation_203 (Activation)     (None, 114, 114, 32) 0           batch_normalization_203[0][0]    
____________________________________________________________________________________________

In [ ]:
len(transfer_model.layers)

376

In [ ]:
for i in range(0, 300):
  transfer_model.layers[i].trainable = False

transfer_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 229, 229, 3) 0                                            
__________________________________________________________________________________________________
conv2d_406 (Conv2D)             (None, 114, 114, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_406 (BatchN (None, 114, 114, 32) 96          conv2d_406[0][0]                 
__________________________________________________________________________________________________
activation_406 (Activation)     (None, 114, 114, 32) 0           batch_normalization_406[0][0]    
____________________________________________________________________________________________

Basic Model

In [ ]:
transfer_model = Sequential([
    pre_trained_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')
])
transfer_model.summary()

Prepare for training

In [ ]:
logdir = '/Users/user/Desktop/b21_proj/logs/image/' + version_name + '/' + datetime.now().strftime('%Y%m%d-%H%M%S')

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

In [ ]:
transfer_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

save_model_dir = '/Users/user/Desktop/b21_proj/Model/' + version_name + '/'                 ## Change the last subdir for different model architectures
checkpoint = tf.keras.callbacks.ModelCheckpoint(save_model_dir, monitor='val_accuracy', verbose=0, save_best_only=True)
earlyStop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=20, min_delta=0.001)

In [ ]:
trf_history = transfer_model.fit(train_generator,
                        epochs=NUM_EPOCHS,
                        verbose=1,
                        batch_size=BATCH_SIZE,
                        validation_data=test_generator,
                        callbacks=[checkpoint, earlyStop],
                        )

In [ ]:
res = transfer_model.evaluate(test_generator)

#Evaluate Model

In [ ]:
test = tf.keras.models.load_model('C:/Users/user/Desktop/b21_proj/Model/' + 'InceptionResNetV2')

In [ ]:
test_sm = tf.saved_model.save(test, 'D:/proj_img/sm_1')

INFO:tensorflow:Assets written to: D:/proj_img/sm_1\assets


In [ ]:
version_name

'InceptionResNetV2_2018_2020'

In [ ]:
preds = transfer_model.predict(test_generator)
preds_list = []

for pred in preds:
  preds_list.append(np.argmax(pred))

report = classification_report(test_generator.classes, preds_list)

#class_name = ["akiec", 'bcc' ,'bkl' ,'df' ,'mel' ,'nv' ,'vasc']
print(report)

With Augmentation


In [ ]:
version_name

'InceptionResNetV2_Aug_Stratify'

In [ ]:
transfer_model = tf.keras.models.load_model('/Users/user/Desktop/b21_proj/Model/' + version_name)

In [ ]:
transfer_model.layers[0].trainable = False
transfer_model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Functio (None, None, None, 1536)  54336736  
_________________________________________________________________
global_average_pooling2d_9 ( (None, 1536)              0         
_________________________________________________________________
dense_14 (Dense)             (None, 512)               786944    
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 7)                 3591      
Total params: 55,127,271
Trainable params: 790,535
Non-trainable params: 54,336,736
_________________________________________________________________


## Converting model to tflite


In [ ]:
model = tf.keras.models.load_model('/Users/user/Desktop/b21_proj/Model/' + version_name)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmplwfytam3\assets


INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmplwfytam3\assets


In [ ]:
TFLITE_MODEL = '/Users/user/Desktop/b21_proj/tflite_Model/' + version_name + '.tflite'
TFLITE_QUANT_MODEL = '/Users/user/Desktop/b21_proj/tflite_Model/' + version_name + '_quant.tflite'

In [ ]:
tflite_model_file = pathlib.Path(TFLITE_MODEL)
tflite_model_file.write_bytes(tflite_model)

190116068

In [ ]:
# Get the concrete function from the Keras model.
run_model = tf.function(lambda x : model(x))

# Save the concrete function.
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype)
)

# Convert the model to standard TensorFlow Lite model
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converted_tflite_model = converter.convert()
open(TFLITE_MODEL, "wb").write(converted_tflite_model)

# Convert the model to quantized version with post-training quantization
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_quant_model = converter.convert()
open(TFLITE_QUANT_MODEL, "wb").write(tflite_quant_model)

49228048

##Run TFLite models


In [ ]:
tflite_interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL)

input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])
print("\n== Output details ==")
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])


== Input details ==
shape: [  1 299 299   3]
type: <class 'numpy.float32'>

== Output details ==
shape: [1 7]
type: <class 'numpy.float32'>


In [ ]:
tflite_interpreter.allocate_tensors()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

In [ ]:
# Read image and decode
img = tf.io.read_file('C:/Users/user/Desktop/b21_proj/Dataset/image_dir/train/mel/_0_39.jpg')
img_tensor = tf.image.decode_image(img)

# Preprocess image
img_size = (299, 299)
img_tensor = tf.image.resize(img_tensor, img_size)
img_tensor = tf.cast(img_tensor, tf.float32)

# Add a batch dimension
input_data = tf.expand_dims(img_tensor, axis=0)

In [ ]:
tflite_interpreter.set_tensor(input_details[0]['index'], input_data)
tflite_interpreter.invoke()

In [ ]:
input_index = tflite_interpreter.get_input_details()[0]['index']
output_index = tflite_interpreter.get_output_details()[0]['index']

predictions = []
test_labels = []

for i in range(len(test_generator)):
  tflite_interpreter.set_tensor(input_index, next(test_generator)[0])
  tflite_interpreter.invoke()
  predictions.append(tflite_interpreter.get_tensor(output_index))
  test_labels.append(next(test_generator)[1])

In [ ]:
i = 0

In [ ]:
print(test_labels[i])
print(predictions[i])
i += 1

In [ ]:
plus_score = 0

for index, pred in enumerate(predictions):
  predict_output = int(np.argmax(pred))
  true_output = int(np.where(test_labels[index]==1)[1])
  if predict_output == true_output:
    plus_score += 1

val_acc = plus_score / len(predictions)
print('val accuracy: {}'.format(val_acc))

In [ ]:
labels = ["akiec", 'bcc' ,'bkl' ,'df' ,'mel' ,'nv' ,'vasc']
top_k_result = 3
_, top_k_indices = tf.math.top_k(predictions, k=top_k_result)
#top_k_indices = np.array(top_k_indices)[0]


for index, pred in enumerate(predictions):
  true_idx = int(np.where(test_labels[index] == 1)[1])
  print('true label: {}'.format(labels[true_idx]))
  for i in range(top_k_result):
    idx = int(top_k_indices[index].numpy()[0][i])
    print('{}, {:.2f}'.format(labels[idx], pred[0][idx] * 100))
  print('\n')

In [ ]:
input_index = tflite_interpreter.get_input_details()[0]['index']
output_index = tflite_interpreter.get_output_details()[0]['index']

predictions = []
test_labels = []

for i in range(0,100):
  tflite_interpreter.set_tensor(input_details, next(test_generator)[0])
  tflite_interpreter.invoke()
  predictions.append(tflite_interpreter.get_tensor(output_index))
  test_labels.append(next(test_generator)[1])

In [ ]:
labels = ["akiec", 'bcc' ,'bkl' ,'df' ,'mel' ,'nv' ,'vasc']
top_k_indices = 3
_, top_k_indices = tf.math.top_k(predictions, k=top_k_result)
